In [1]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

In [2]:
!ls -l ../data/

total 580596
-rw------- 1 v.bugaevskii v.bugaevskii    904039 Apr 17 11:12 appl.csv
-rw------- 1 v.bugaevskii v.bugaevskii   6577695 Apr 17 11:12 aum.csv
-rw------- 1 v.bugaevskii v.bugaevskii  91636746 Apr 17 11:12 balance.csv
-rw------- 1 v.bugaevskii v.bugaevskii   1110765 Apr 17 11:12 client.csv
-rw------- 1 v.bugaevskii v.bugaevskii   6686165 Apr 17 11:12 com.csv
-rw------- 1 v.bugaevskii v.bugaevskii   7499804 Apr 17 11:12 deals.csv
-rw------- 1 v.bugaevskii v.bugaevskii     18979 Apr 17 11:12 dict_mcc.csv
-rw------- 1 v.bugaevskii v.bugaevskii   2322389 Apr 17 11:12 funnel.csv
-rw------- 1 v.bugaevskii v.bugaevskii  10553763 Apr 17 11:12 payments.csv
-rw------- 1 v.bugaevskii v.bugaevskii 467193115 Apr 17 11:12 trxn.csv


In [3]:
from pathlib import Path

In [4]:
path_df = Path('../data').rglob('*.csv')
path_df = sorted(path_df)

for path in tqdm(path_df):
    df_name = 'df_' + path.name.rsplit('.', 1)[0]
    vars()[df_name] = pd.read_csv(path)
    print(f'{path} saved to {df_name}; {df_name}.shape = {vars()[df_name].shape}')

  0%|          | 0/10 [00:00<?, ?it/s]

../data/appl.csv saved to df_appl; df_appl.shape = (12030, 6)
../data/aum.csv saved to df_aum; df_aum.shape = (117392, 4)
../data/balance.csv saved to df_balance; df_balance.shape = (1194684, 9)
../data/client.csv saved to df_client; df_client.shape = (21498, 8)
../data/com.csv saved to df_com; df_com.shape = (113055, 10)
../data/deals.csv saved to df_deals; df_deals.shape = (109016, 8)
../data/dict_mcc.csv saved to df_dict_mcc; df_dict_mcc.shape = (915, 3)
../data/funnel.csv saved to df_funnel; df_funnel.shape = (21498, 16)
../data/payments.csv saved to df_payments; df_payments.shape = (188068, 4)


/home/v.bugaevskii/.conda/envs/idao2021/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


../data/trxn.csv saved to df_trxn; df_trxn.shape = (3035705, 11)


In [5]:
df_balance.sort_values(by='month_end_dt', inplace=True)

mask = np.isclose(df_balance[['min_bal_sum_rur', 'max_bal_sum_rur']].values, 0).all(axis=1)
df_balance = df_balance[~mask]
df_balance['prod_group_name'] = df_balance['prod_group_name']\
    .replace('Credit card 120 days', 'Credit card other')
df_balance = df_balance[~df_balance['prod_group_name'].isin({'Car loans', 'Mortgage', 'Technical cards'})]

df_balance

,client_id,crncy_cd,eop_bal_sum_rur,min_bal_sum_rur,max_bal_sum_rur,avg_bal_sum_rur,month_end_dt,prod_cat_name,prod_group_name
932238,-2552615764267493540,810.0,9.0,2.0,21.0,11.0,2018-09-30,KN,NaN
392953,-4462132791141929311,810.0,218.0,217.0,218.0,217.0,2018-09-30,DEPOSITS,Time deposits
392716,9003415646143201738,810.0,141.0,141.0,141.0,141.0,2018-09-30,DEPOSITS,Time deposits
91958,-8659341400857604408,810.0,5.0,5.0,5.0,5.0,2018-09-30,CURRENT ACCOUNTS,Cash on demand
393135,-4211849792309067622,810.0,748.0,744.0,748.0,744.0,2018-09-30,DEPOSITS,Time deposits
...,...,...,...,...,...,...,...,...,...
893887,5948443102508817040,810.0,-912.0,-927.0,-912.0,-921.0,2019-08-31,KN,PILS
893888,5948443102508817040,810.0,-64.0,-73.0,-64.0,-69.0,2019-08-31,KN,PILS
151541,-1016172887860131818,810.0,1014.0,1002.0,1014.0,1006.0,2019-08-31,CURRENT ACCOUNTS,Cash on demand
893745,1416156577753380608,810.0,-2679.0,-2716.0,-2679.0,-2701.0,2019-08-31,KN,PILS


In [6]:
df_grouped = df_balance.groupby(['client_id', 'prod_group_name']).tail(1)
df_grouped.head()

,client_id,crncy_cd,eop_bal_sum_rur,min_bal_sum_rur,max_bal_sum_rur,avg_bal_sum_rur,month_end_dt,prod_cat_name,prod_group_name
390009,-806113865819871747,810.0,0.0,1166.0,1173.0,1167.0,2018-09-30,DEPOSITS,Time deposits
390968,-6121513898976461502,810.0,0.0,88.0,88.0,88.0,2018-09-30,DEPOSITS,Time deposits
398335,821538400184123281,810.0,0.0,86.0,86.0,86.0,2018-09-30,DEPOSITS,Time deposits
382952,6646356236632481327,810.0,0.0,736.0,739.0,739.0,2018-09-30,DEPOSITS,Time deposits
95994,6245520333833780039,810.0,0.0,56.0,56.0,56.0,2018-09-30,CURRENT ACCOUNTS,Cash on demand


In [7]:
df_features_pull = []

for values in ['eop_bal_sum_rur', 'min_bal_sum_rur', 'max_bal_sum_rur', 'avg_bal_sum_rur']:
    df_features_ = pd.pivot_table(
        df_grouped,
        values=values,
        index='client_id', 
        columns='prod_group_name',
    )
    df_features_.columns = df_features_.columns.str.replace(' ', '_')
    df_features_ = df_features_.add_prefix(values + '_').add_suffix('_last_month')
    df_features_pull.append(df_features_)

df_features = pd.concat(df_features_pull, axis=1)
df_features.head()

prod_group_name,eop_bal_sum_rur_Cash_on_demand_last_month,eop_bal_sum_rur_Credit_card_other_last_month,eop_bal_sum_rur_Debit_cards_last_month,eop_bal_sum_rur_Open_card_credit_card_last_month,eop_bal_sum_rur_PILS_last_month,eop_bal_sum_rur_Salary_cards_last_month,eop_bal_sum_rur_Time_deposits_last_month,min_bal_sum_rur_Cash_on_demand_last_month,min_bal_sum_rur_Credit_card_other_last_month,min_bal_sum_rur_Debit_cards_last_month,...,max_bal_sum_rur_PILS_last_month,max_bal_sum_rur_Salary_cards_last_month,max_bal_sum_rur_Time_deposits_last_month,avg_bal_sum_rur_Cash_on_demand_last_month,avg_bal_sum_rur_Credit_card_other_last_month,avg_bal_sum_rur_Debit_cards_last_month,avg_bal_sum_rur_Open_card_credit_card_last_month,avg_bal_sum_rur_PILS_last_month,avg_bal_sum_rur_Salary_cards_last_month,avg_bal_sum_rur_Time_deposits_last_month
client_id,,,,,,,,,,,,,,,,,,,,,
-9220236243053692422,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
-9220233431709087652,0.0,NaN,NaN,0.0,-674.0,0.0,NaN,3.0,NaN,NaN,...,-674.0,13.0,NaN,3.0,NaN,NaN,-11.0,-677.0,2.0,NaN
-9219699286371310531,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
-9218923580775423458,NaN,NaN,NaN,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
-9218871523310554579,NaN,NaN,NaN,NaN,NaN,41.0,NaN,NaN,NaN,NaN,...,NaN,78.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN


In [8]:
df_features.isna().sum(axis=0) / df_features.shape[0]

prod_group_name
eop_bal_sum_rur_Cash_on_demand_last_month           0.666303
eop_bal_sum_rur_Credit_card_other_last_month        0.911204
eop_bal_sum_rur_Debit_cards_last_month              0.639782
eop_bal_sum_rur_Open_card_credit_card_last_month    0.879899
eop_bal_sum_rur_PILS_last_month                     0.708435
eop_bal_sum_rur_Salary_cards_last_month             0.267310
eop_bal_sum_rur_Time_deposits_last_month            0.928493
min_bal_sum_rur_Cash_on_demand_last_month           0.666303
min_bal_sum_rur_Credit_card_other_last_month        0.911204
min_bal_sum_rur_Debit_cards_last_month              0.639782
min_bal_sum_rur_Open_card_credit_card_last_month    0.879899
min_bal_sum_rur_PILS_last_month                     0.708435
min_bal_sum_rur_Salary_cards_last_month             0.267310
min_bal_sum_rur_Time_deposits_last_month            0.928493
max_bal_sum_rur_Cash_on_demand_last_month           0.666303
max_bal_sum_rur_Credit_card_other_last_month        0.911204
max_bal_

In [9]:
counts = df_features.isna().sum(axis=0) / df_features.shape[0]
counts = counts[counts >= 0.95]
counts.index

Index([], dtype='object', name='prod_group_name')

In [10]:
df_grouped = df_balance.groupby(['client_id', 'prod_group_name']).agg({
    'min_bal_sum_rur': 'min',
    'max_bal_sum_rur': 'max',
}).reset_index()

In [11]:
df_features_pull = []

for values in ['min_bal_sum_rur', 'max_bal_sum_rur']:
    df_features_ = pd.pivot_table(
        df_grouped,
        values=values,
        index='client_id', 
        columns='prod_group_name',
    )
    
    df_features_.columns = df_features_.columns.str.replace(' ', '_')
    df_features_ = df_features_.add_prefix(values + '_')
    df_features_pull.append(df_features_)

df_features_ = pd.concat(df_features_pull, axis=1)
df_features_.head()

prod_group_name,min_bal_sum_rur_Cash_on_demand,min_bal_sum_rur_Credit_card_other,min_bal_sum_rur_Debit_cards,min_bal_sum_rur_Open_card_credit_card,min_bal_sum_rur_PILS,min_bal_sum_rur_Salary_cards,min_bal_sum_rur_Time_deposits,max_bal_sum_rur_Cash_on_demand,max_bal_sum_rur_Credit_card_other,max_bal_sum_rur_Debit_cards,max_bal_sum_rur_Open_card_credit_card,max_bal_sum_rur_PILS,max_bal_sum_rur_Salary_cards,max_bal_sum_rur_Time_deposits
client_id,,,,,,,,,,,,,,
-9220236243053692422,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN
-9220233431709087652,0.0,NaN,NaN,-182.0,-746.0,0.0,NaN,78.0,NaN,NaN,2.0,-76.0,76.0,NaN
-9219699286371310531,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
-9218923580775423458,NaN,NaN,NaN,-27.0,NaN,NaN,NaN,NaN,NaN,NaN,67.0,NaN,NaN,NaN
-9218871523310554579,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,78.0,NaN


In [12]:
counts = df_features_.isna().sum(axis=0) / df_features_.shape[0]
counts = counts[counts >= 0.95]
counts.index

Index([], dtype='object', name='prod_group_name')

In [13]:
df_features = pd.concat([df_features, df_features_], axis=1)
df_features.head()

prod_group_name,eop_bal_sum_rur_Cash_on_demand_last_month,eop_bal_sum_rur_Credit_card_other_last_month,eop_bal_sum_rur_Debit_cards_last_month,eop_bal_sum_rur_Open_card_credit_card_last_month,eop_bal_sum_rur_PILS_last_month,eop_bal_sum_rur_Salary_cards_last_month,eop_bal_sum_rur_Time_deposits_last_month,min_bal_sum_rur_Cash_on_demand_last_month,min_bal_sum_rur_Credit_card_other_last_month,min_bal_sum_rur_Debit_cards_last_month,...,min_bal_sum_rur_PILS,min_bal_sum_rur_Salary_cards,min_bal_sum_rur_Time_deposits,max_bal_sum_rur_Cash_on_demand,max_bal_sum_rur_Credit_card_other,max_bal_sum_rur_Debit_cards,max_bal_sum_rur_Open_card_credit_card,max_bal_sum_rur_PILS,max_bal_sum_rur_Salary_cards,max_bal_sum_rur_Time_deposits
client_id,,,,,,,,,,,,,,,,,,,,,
-9220236243053692422,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN
-9220233431709087652,0.0,NaN,NaN,0.0,-674.0,0.0,NaN,3.0,NaN,NaN,...,-746.0,0.0,NaN,78.0,NaN,NaN,2.0,-76.0,76.0,NaN
-9219699286371310531,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
-9218923580775423458,NaN,NaN,NaN,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,67.0,NaN,NaN,NaN
-9218871523310554579,NaN,NaN,NaN,NaN,NaN,41.0,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,78.0,NaN


In [14]:
df_features.to_csv('balance_features.csv', index=True)

In [15]:
df_features.shape

(11915, 42)